In [13]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# set executable path in the next cell 
# then, set up the URL NASA Mars News for scraping 

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Downloading: 100%|█████████████████| 8.15M/8.15M [00:00<00:00, 30.3MB/s]


In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
# searching for elements with a specific combination of tag (div) and attriute (list_text) 
# telling browser to wait one second before searching for components 
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# setting up the HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
# parent element: this holds all the other elements within it, we reference this when we want to filter search results even further 
slide_elem = news_soup.select_one('div.list_text')

In [5]:
# scraping the data
# chained ".find" onto our previously assigned variable, slide_elem
# = this variable holds a ton of information, look inside that info to find this specific data
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA's Perseverance Rover Goes Through Trials by Fire, Ice, Light and Sound</div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`
# .get_text() only the text of the element is returned 
# in this code, we have created a new variable for the title, added the get_text() method and we're searching within the parent element for the title
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Perseverance Rover Goes Through Trials by Fire, Ice, Light and Sound"

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"The agency's new Mars rover is put through a series of tests in vacuum chambers, acoustic chambers and more to get ready for the Red Planet."

### Featured Images 

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [11]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [12]:
# Use the base URL to create an absolute URL

# this way, when JPL updates its image page, our code will still pull the most recent image 
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [14]:
# scrpaing the entire table with Pandas' read.html() fxn


# creating a new DF from the HTML table 
# read_html() specifically searches for and returns a list of tables found in the HTML 
df = pd.read_html('https://galaxyfacts-mars.com')[0]

# assign columns to the new DF for additional clarity 
df.columns=['description', 'Mars', 'Earth']

# we're turning the description column into the DF's index
# inplace=True means that the updated index will remain in place w/out having to readdign the DF to a new variable
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
# convert DF back into HTML-ready code using the .to_html() fxn 
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [16]:
# end the session
browser.quit()